# EP2 - Mandelbrot set CUDA+OMPI
###### Notebook baseado no dos miniEPs

Preencha o nome dos 5 membros do seu grupo na tabela abaixo:

| Nome | NUSP |
|------|------|
| Caio Andrade | 9797232 |
| Caio Fontes | 10692061 |
| Eduardo Laurentino | 8988212 |
| Thiago Teixeira | 10736987 |
| Washington Meireles | 10737157 |

In [ ]:
] up

In [ ]:
] st

## Tarefa 3 - Apresentação dos resultados

### Verificando se os programas estão funcionais:

In [ ]:
;make 

In [ ]:
; ./mandelbrot_seq -2.5 1.5 -2.0 2.0 0

In [ ]:
; ./mandelbrot_pth -2.5 1.5 -2.0 2.0 1 1

In [ ]:
;./mandelbrot_omp -2.5 1.5 -2.0 2.0 1 1

In [ ]:
;./mandelbrot_cuda -2.5 1.5 -2.0 2.0 1 1

In [ ]:
using DataFrames, Query, StatsPlots, Statistics

function measure_mandelbrot(size, f, typ; thread = 0)
    if f==0 par = `-2.5 1.5 -2.0 2.0` #Full Picture
    elseif f==1 par = `-0.8 -0.7 0.05 0.15` #Seahorse Valley
    elseif f==2 par = `0.175 0.375 -0.1 0.1` #Elephant Valley
    elseif f==3 par = `-0.188 -0.012 0.554 0.754` #Triple Spiral Valley
    end                
    if thread == 0 
        results = parse.(Float64,
            chomp(read(`./$typ $par $size`, String)))
    else
        results = parse.(Float64,
            chomp(read(`./$typ $par $size $thread`, String)))
    end
        
    return DataFrame(size = size,
        f = f,
        threads = thread,
        duration = results[1])
end

A função `run_experiments` recebe os mesmos parâmetros `size`, `f`, `method` e `threads`, e um parâmetro adicional `repetitions`, com o número de repetições de cada experimento com um dado número de `threads`. A função devolve um `DataFrame` com todos os experimentos.

In [ ]:
function run_experiments(size, f, method, repetitions; threads = [])
    run(`make $method`)
    
    results = DataFrame(size = Int[],
        f = Int[],
        threads = Int[],
        duration = Float64[])  
    
    if threads == []
        for r in 1:repetitions
            for s in size
                append!(results,
                    measure_mandelbrot(s, f, method))    
            end
        end
    else
        for t in threads
            for s in size
                for r in 1:repetitions
                    append!(results,
                        measure_mandelbrot(s, f, method, thread = t))
                    end
                end
            end
        end
    return results
end

A função `parse_results` recebe um `DataFrame` de resultados, produzido pela função `run_experiments`. A função devolve um `DataFrame` com a média e o intervalo de confiança da média a 95% das estimativas e dos tempos de execução, agrupados por número de threads.

In [ ]:
function parse_results(results)
    parsed_results = results |>
                    @groupby({_.threads,_.size}) |>
                    @map({threads = key(_).threads,
                          size = _.size[1],
                          mean_duration = mean(_.duration),
                          ci_duration = 1.96 * std(_.duration)}) |>
                    DataFrame
    
    return parsed_results
end

## Realizando os experimentos:

### DataFrames:

Parametros a serem utilizados(usamos os mesmos do script `run_measurements.sh`):

In [ ]:
size = [2 ^ i for i in 4:13] #resolucao
thread = [2 ^ i for i in 0:5]
repetitions = 10;

#### Sequencial:

In [ ]:
fileName = "mandelbrot_seq"

In [ ]:
#Full picture
results = run_experiments(size, 0, fileName, repetitions)
seq_full = parse_results(results)

In [ ]:
#Seahorse valley
results = run_experiments(size, 1, fileName , repetitions)
seq_seahorse = parse_results(results)

In [ ]:
#Elephant valley
results = run_experiments(size, 2, fileName, repetitions)
seq_elephant = parse_results(results)

In [ ]:
#Triple spiral
results = run_experiments(size, 3, fileName, repetitions)
seq_tripleSpiral = parse_results(results)

#### PThreads:

In [ ]:
fileName = "mandelbrot_pth"

In [ ]:
#Full picture
results = run_experiments(size, 0, fileName, repetitions, threads=thread)
pth_full = parse_results(results)

In [ ]:
#Seahorse valley
results = run_experiments(size, 1, fileName, repetitions, threads=thread)
pth_seahorse = parse_results(results)

In [ ]:
#Elephant valley
results = run_experiments(size, 2, fileName, repetitions, threads=thread)
pth_elephant = parse_results(results)

In [ ]:
#Triple spiral
results = run_experiments(size, 3, fileName, repetitions, threads=thread)
pth_tripleSpiral = parse_results(results)

#### OpenMP:

In [ ]:
fileName = "mandelbrot_omp"

In [ ]:
#Full picture
results = run_experiments(size, 0, fileName, repetitions, threads=thread)
omp_full = parse_results(results)

In [ ]:
#Seahores Valley
results = run_experiments(size, 1, fileName, repetitions, threads=thread)
omp_seahorse = parse_results(results)

In [ ]:
#Elephant Valley
results = run_experiments(size, 2, fileName, repetitions, threads=thread)
omp_elephant = parse_results(results)

In [ ]:
#Triple spiral
results = run_experiments(size, 3, fileName, repetitions, threads=thread)
omp_tripleSpiral = parse_results(results)

#### CUDA:

In [ ]:
fileName = "mandelbrot_cuda"

In [ ]:
#Full picture
results = run_experiments(size, 0, fileName, repetitions, threads=thread)
cuda_full = parse_results(results)

In [ ]:
#Seahores Valley
results = run_experiments(size, 1, fileName, repetitions, threads=thread)
cuda_seahorse = parse_results(results)

In [ ]:
#Elephant Valley
results = run_experiments(size, 2, fileName, repetitions, threads=thread)
cuda_elephant = parse_results(results)

In [ ]:
#Triple spiral
results = run_experiments(size, 3, fileName, repetitions, threads=thread)
cuda_tripleSpiral = parse_results(results)

#### Exportando em .csv:

In [ ]:
using CSV
function save_csv_results(parsed_results, name)
    CSV.write(string(name, ".csv"), parsed_results)
end

In [ ]:
#Sequencial
save_csv_results(seq_full, "seq_full")
save_csv_results(seq_seahorse, "seq_seahorse")
save_csv_results(seq_elephant, "seq_elephant")
save_csv_results(seq_tripleSpiral, "seq_tripleSpiral")

In [ ]:
#Pthreads
save_csv_results(pth_full, "pth_full")
save_csv_results(pth_seahorse, "pth_seahorse")
save_csv_results(pth_elephant, "pth_elephant")
save_csv_results(pth_tripleSpiral, "pth_tripleSpiral")

In [ ]:
#OpenMP
save_csv_results(omp_full, "omp_full")
save_csv_results(omp_seahorse, "omp_seahorse")
save_csv_results(omp_elephant, "omp_elephant")
save_csv_results(omp_tripleSpiral, "omp_tripleSpiral")

In [ ]:
#CUDA
save_csv_results(cuda_full, "cuda_full")
save_csv_results(cuda_seahorse, "cuda_seahorse")
save_csv_results(cuda_elephant, "cuda_elephant")
save_csv_results(cuda_tripleSpiral, "cuda_tripleSpiral")

#### Importando .csv:

In [ ]:
function read_csv_results(filename)
    return CSV.read(filename)
end

In [ ]:
#Sequencial
seq_full = read_csv_results("seq_full.csv")
seq_seahorse = read_csv_results("seq_seahorse.csv")
seq_elephant = read_csv_results("seq_elephant.csv")
seq_tripleSpiral = read_csv_results("seq_tripleSpiral.csv")

In [ ]:
#Pthreads
pth_full = read_csv_results("pth_full.csv")
pth_seahorse = read_csv_results("pth_seahorse.csv")
pth_elephant = read_csv_results("pth_elephant.csv")
pth_tripleSpiral = read_csv_results("pth_tripleSpiral.csv")

In [ ]:
#OpenMP
omp_full = read_csv_results("omp_full.csv")
omp_seahorse = read_csv_results("omp_seahorse.csv")
omp_elephant = read_csv_results("omp_elephant.csv")
omp_tripleSpiral = read_csv_results("omp_tripleSpiral.csv")

In [ ]:
#CUDA
cuda_full = read_csv_results("cuda_full.csv")
cuda_seahorse = read_csv_results("cuda_seahorse.csv")
cuda_elephant = read_csv_results("cuda_elephant.csv")
cuda_tripleSpiral = read_csv_results("cuda_tripleSpiral.csv")

## Gráficos:

Função para gerar os gráficos:

In [ ]:
using StatsPlots
function plot_results(x, y1, series_label1, yerror1,
                        y2, series_label2, yerror2,
                        y3, series_label3, yerror3,
                        y4, series_label4, yerror4,
                        y5, series_label5, yerror5,
                        y6, series_label6, yerror6)    
    p = plot(x,        
            y1,
            yerror = yerror1,
            alpha = 0.9,
            labels = series_label1,
            xlabel = "size",
            color = "red",
            lw = 1,
            legend = :topleft)
    
    if y2 != []
            plot!(x,        
            y2,
            yerror = yerror2,
            alpha = 0.9,
            labels = series_label2,
            color = "blue",
            lw = 1,
            legend = :topleft)
    end
    if y3 != []
            plot!(x,        
            y3,
            yerror = yerror3,
            alpha = 0.9,
            labels = series_label3,
            color = "green",
            lw = 1,
            legend = :topleft)
    end
    if y4 != []
            plot!(x,        
            y4,
            yerror = yerror4,
            alpha = 0.9,
            labels = series_label4,
            color = "black",
            lw = 1,
            legend = :topleft)
    end
    if y5 != []
            plot!(x,        
            y5,
            yerror = yerror5,
            alpha = 0.9,
            labels = series_label5,
            color = "purple",
            lw = 1,
            legend = :topleft)
    end
    if y6 != []
            plot!(x,        
            y6,
            yerror = yerror6,
            alpha = 0.9,
            labels = series_label6,
            color = "orange",
            lw = 1,
            legend = :topleft)
    end
    return p
end


### Full picture:
#### Sequencial:

In [ ]:
plot_results(seq_full.size, seq_full.mean_duration, "Sequencial", seq_full.ci_duration,
                [], [], [], [], [],
                [], [], [], [], [],
                [], [], [], [], [])

#### Pthreads:

In [ ]:
pth_1 = filter(row -> row[:threads] == 1, pth_full)
pth_2 = filter(row -> row[:threads] == 2, pth_full)
pth_4 = filter(row -> row[:threads] == 4, pth_full)
pth_8 = filter(row -> row[:threads] == 8, pth_full)
pth_16 = filter(row -> row[:threads] == 16, pth_full)
pth_32 = filter(row -> row[:threads] == 32, pth_full)

In [ ]:
plot_results(pth_1.size, pth_1.mean_duration, "1 thread", pth_1.ci_duration,
            pth_2.mean_duration, "2 threads", pth_2.ci_duration,
            pth_4.mean_duration, "4 threads", pth_4.ci_duration,
            pth_8.mean_duration, "8 threads", pth_8.ci_duration,
            pth_16.mean_duration, "16 threads", pth_16.ci_duration,
            pth_32.mean_duration, "32 threads", pth_32.ci_duration)


#### OpenMP:

In [ ]:
omp_1 = filter(row -> row[:threads] == 1, omp_full)
omp_2 = filter(row -> row[:threads] == 2, omp_full)
omp_4 = filter(row -> row[:threads] == 4, omp_full)
omp_8 = filter(row -> row[:threads] == 8, omp_full)
omp_16 = filter(row -> row[:threads] == 16, omp_full)
omp_32 = filter(row -> row[:threads] == 32, omp_full)

In [ ]:
plot_results(omp_1.size, omp_1.mean_duration, "1 thread", omp_1.ci_duration,
            omp_2.mean_duration, "2 threads", omp_2.ci_duration,
            omp_4.mean_duration, "4 threads", omp_4.ci_duration,
            omp_8.mean_duration, "8 threads", omp_8.ci_duration,
            omp_16.mean_duration, "16 threads", omp_16.ci_duration,
            omp_32.mean_duration, "32 threads", omp_32.ci_duration)

### Seahorse Valley:
#### Sequencial:

In [ ]:
plot_results(seq_seahorse.size, seq_seahorse.mean_duration, "Sequencial", seq_seahorse.ci_duration,
                [], [], [], [], [],
                [], [], [], [], [],
                [], [], [], [], [])

#### Pthreads:

In [ ]:
pth_1 = filter(row -> row[:threads] == 1, pth_seahorse)
pth_2 = filter(row -> row[:threads] == 2, pth_seahorse)
pth_4 = filter(row -> row[:threads] == 4, pth_seahorse)
pth_8 = filter(row -> row[:threads] == 8, pth_seahorse)
pth_16 = filter(row -> row[:threads] == 16, pth_seahorse)
pth_32 = filter(row -> row[:threads] == 32, pth_seahorse)

In [ ]:
plot_results(pth_1.size, pth_1.mean_duration, "1 thread", pth_1.ci_duration,
            pth_2.mean_duration, "2 threads", pth_2.ci_duration,
            pth_4.mean_duration, "4 threads", pth_4.ci_duration,
            pth_8.mean_duration, "8 threads", pth_8.ci_duration,
            pth_16.mean_duration, "16 threads", pth_16.ci_duration,
            pth_32.mean_duration, "32 threads", pth_32.ci_duration)

#### OpenMP:

In [ ]:
omp_1 = filter(row -> row[:threads] == 1, omp_seahorse)
omp_2 = filter(row -> row[:threads] == 2, omp_seahorse)
omp_4 = filter(row -> row[:threads] == 4, omp_seahorse)
omp_8 = filter(row -> row[:threads] == 8, omp_seahorse)
omp_16 = filter(row -> row[:threads] == 16, omp_seahorse)
omp_32 = filter(row -> row[:threads] == 32, omp_seahorse)

In [ ]:
plot_results(omp_1.size, omp_1.mean_duration, "1 thread", omp_1.ci_duration,
            omp_2.mean_duration, "2 threads", omp_2.ci_duration,
            omp_4.mean_duration, "4 threads", omp_4.ci_duration,
            omp_8.mean_duration, "8 threads", omp_8.ci_duration,
            omp_16.mean_duration, "16 threads", omp_16.ci_duration,
            omp_32.mean_duration, "32 threads", omp_32.ci_duration)

#### CUDA:

In [ ]:
cuda_1 = filter(row -> row[:threads] == 1, cuda_seahorse)
cuda_2 = filter(row -> row[:threads] == 2, cuda_seahorse)
cuda_4 = filter(row -> row[:threads] == 4, cuda_seahorse)
cuda_8 = filter(row -> row[:threads] == 8, cuda_seahorse)
cuda_16 = filter(row -> row[:threads] == 16, cuda_seahorse)
cuda_32 = filter(row -> row[:threads] == 32, cuda_seahorse)

In [ ]:
plot_results(cuda_1.size, omp_1.mean_duration, "1 thread", cuda_1.ci_duration,
            cuda_2.mean_duration, "2 threads", cuda_2.ci_duration,
            cuda_4.mean_duration, "4 threads", cuda_4.ci_duration,
            cuda_8.mean_duration, "8 threads", cuda_8.ci_duration,
            cuda_16.mean_duration, "16 threads", cuda_16.ci_duration,
            cuda_32.mean_duration, "32 threads", cuda_32.ci_duration)

### Elephant Valley:
#### Sequencial:

In [ ]:
plot_results(seq_elephant.size, seq_elephant.mean_duration, "Sequencial", seq_elephant.ci_duration,
                [], [], [], [], [],
                [], [], [], [], [],
                [], [], [], [], [])

#### Pthreads:

In [ ]:
pth_1 = filter(row -> row[:threads] == 1, pth_elephant)
pth_2 = filter(row -> row[:threads] == 2, pth_elephant)
pth_4 = filter(row -> row[:threads] == 4, pth_elephant)
pth_8 = filter(row -> row[:threads] == 8, pth_elephant)
pth_16 = filter(row -> row[:threads] == 16, pth_elephant)
pth_32 = filter(row -> row[:threads] == 32, pth_elephant)

In [ ]:
plot_results(pth_1.size, pth_1.mean_duration, "1 thread", pth_1.ci_duration,
            pth_2.mean_duration, "2 threads", pth_2.ci_duration,
            pth_4.mean_duration, "4 threads", pth_4.ci_duration,
            pth_8.mean_duration, "8 threads", pth_8.ci_duration,
            pth_16.mean_duration, "16 threads", pth_16.ci_duration,
            pth_32.mean_duration, "32 threads", pth_32.ci_duration)

#### OpenMP:

In [ ]:
omp_1 = filter(row -> row[:threads] == 1, omp_elephant)
omp_2 = filter(row -> row[:threads] == 2, omp_elephant)
omp_4 = filter(row -> row[:threads] == 4, omp_elephant)
omp_8 = filter(row -> row[:threads] == 8, omp_elephant)
omp_16 = filter(row -> row[:threads] == 16, omp_elephant)
omp_32 = filter(row -> row[:threads] == 32, omp_elephant)

In [ ]:
plot_results(omp_1.size, omp_1.mean_duration, "1 thread", omp_1.ci_duration,
            omp_2.mean_duration, "2 threads", omp_2.ci_duration,
            omp_4.mean_duration, "4 threads", omp_4.ci_duration,
            omp_8.mean_duration, "8 threads", omp_8.ci_duration,
            omp_16.mean_duration, "16 threads", omp_16.ci_duration,
            omp_32.mean_duration, "32 threads", omp_32.ci_duration)

#### CUDA:

In [ ]:
cuda_1 = filter(row -> row[:threads] == 1, cuda_seahorse)
cuda_2 = filter(row -> row[:threads] == 2, cuda_seahorse)
cuda_4 = filter(row -> row[:threads] == 4, cuda_seahorse)
cuda_8 = filter(row -> row[:threads] == 8, cuda_seahorse)
cuda_16 = filter(row -> row[:threads] == 16, cuda_seahorse)
cuda_32 = filter(row -> row[:threads] == 32, cuda_seahorse)

In [ ]:
plot_results(cuda_1.size, omp_1.mean_duration, "1 thread", cuda_1.ci_duration,
            cuda_2.mean_duration, "2 threads", cuda_2.ci_duration,
            cuda_4.mean_duration, "4 threads", cuda_4.ci_duration,
            cuda_8.mean_duration, "8 threads", cuda_8.ci_duration,
            cuda_16.mean_duration, "16 threads", cuda_16.ci_duration,
            cuda_32.mean_duration, "32 threads", cuda_32.ci_duration)

### Triple Spiral Valley
#### Sequencial:

In [ ]:
plot_results(seq_tripleSpiral.size, seq_tripleSpiral.mean_duration, "Sequencial", seq_tripleSpiral.ci_duration,
                [], [], [], [], [],
                [], [], [], [], [],
                [], [], [], [], [])

#### Pthreads:

In [ ]:
pth_1 = filter(row -> row[:threads] == 1, pth_tripleSpiral)
pth_2 = filter(row -> row[:threads] == 2, pth_tripleSpiral)
pth_4 = filter(row -> row[:threads] == 4, pth_tripleSpiral)
pth_8 = filter(row -> row[:threads] == 8, pth_tripleSpiral)
pth_16 = filter(row -> row[:threads] == 16, pth_tripleSpiral)
pth_32 = filter(row -> row[:threads] == 32, pth_tripleSpiral)

In [ ]:
plot_results(pth_1.size, pth_1.mean_duration, "1 thread", pth_1.ci_duration,
            pth_2.mean_duration, "2 threads", pth_2.ci_duration,
            pth_4.mean_duration, "4 threads", pth_4.ci_duration,
            pth_8.mean_duration, "8 threads", pth_8.ci_duration,
            pth_16.mean_duration, "16 threads", pth_16.ci_duration,
            pth_32.mean_duration, "32 threads", pth_32.ci_duration)

#### OpenMP:

In [ ]:
omp_1 = filter(row -> row[:threads] == 1, omp_tripleSpiral)
omp_2 = filter(row -> row[:threads] == 2, omp_tripleSpiral)
omp_4 = filter(row -> row[:threads] == 4, omp_tripleSpiral)
omp_8 = filter(row -> row[:threads] == 8, omp_tripleSpiral)
omp_16 = filter(row -> row[:threads] == 16, omp_tripleSpiral)
omp_32 = filter(row -> row[:threads] == 32, omp_tripleSpiral)

In [ ]:
plot_results(omp_1.size, omp_1.mean_duration, "1 thread", omp_1.ci_duration,
            omp_2.mean_duration, "2 threads", omp_2.ci_duration,
            omp_4.mean_duration, "4 threads", omp_4.ci_duration,
            omp_8.mean_duration, "8 threads", omp_8.ci_duration,
            omp_16.mean_duration, "16 threads", omp_16.ci_duration,
            omp_32.mean_duration, "32 threads", omp_32.ci_duration)

#### CUDA:

In [ ]:
cuda_1 = filter(row -> row[:threads] == 1, cuda_seahorse)
cuda_2 = filter(row -> row[:threads] == 2, cuda_seahorse)
cuda_4 = filter(row -> row[:threads] == 4, cuda_seahorse)
cuda_8 = filter(row -> row[:threads] == 8, cuda_seahorse)
cuda_16 = filter(row -> row[:threads] == 16, cuda_seahorse)
cuda_32 = filter(row -> row[:threads] == 32, cuda_seahorse)

In [ ]:
plot_results(cuda_1.size, omp_1.mean_duration, "1 thread", cuda_1.ci_duration,
            cuda_2.mean_duration, "2 threads", cuda_2.ci_duration,
            cuda_4.mean_duration, "4 threads", cuda_4.ci_duration,
            cuda_8.mean_duration, "8 threads", cuda_8.ci_duration,
            cuda_16.mean_duration, "16 threads", cuda_16.ci_duration,
            cuda_32.mean_duration, "32 threads", cuda_32.ci_duration)